# **May 22nd**
## Testing Fault Tolerance of a GraphCodeBERT-based Buffer Overflow CWE Classification model, by injecting bit-flip faults into the model weight parameters.

##Update:
- Using Ratnaker's new dataset

# **Code Update (24th March):**
* Changed CWEs to Ratnaker's recommendations
* Exploring bit flip injections into different layers other than classifier head
* Flip exponent bits instead of sign bits
* use DFMIT and Defor for performance metrics

## Purpose of the script:
1. Train a GraphCodeBERT-based model to classify code snippets into different CWE types (specifically those related to buffer overflows).

2. Introduce bit-flip noise into the model weights post-training, prior to inference on unseen test data.

3. Evaluate how this noise affects the model's accuracy and robustness.

---

Installing ML and NLP-related libraries, mainly from hugging face

In [1]:
!pip install datasets
!pip install transformers
!pip install accelerate -U
!pip install transformers[torch]
!pip install wandb

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 113.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 85.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 50.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 42.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 103.2 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjitl

In [2]:
!pip uninstall -y transformers
!pip install transformers --upgrade --quiet
!pip show transformers


Found existing installation: transformers 4.52.4
Uninstalling transformers-4.52.4:
  Successfully uninstalled transformers-4.52.4
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.5/10.5 MB 88.8 MB/s eta 0:00:00
Name: transformers
Version: 4.52.4
Summary: State-of-the-art Machine Learning for JAX, PyTorch and TensorFlow
Home-page: https://github.com/huggingface/transformers
Author: The Hugging Face team (past and future) with the help of all our contributors (https://github.com/huggingface/transformers/graphs/contributors)
Author-email: transformers@huggingface.co
License: Apache 2.0 License
Location: /usr/local/lib/python3.11/dist-packages
Requires: filelock, huggingface-hub, numpy, packaging, pyyaml, regex, requests, safetensors, tokenizers, tqdm
Required-by: peft, sentence-transformers


In [3]:
from tqdm import tqdm, trange
import multiprocessing

from torch.optim import AdamW  # UPDATED
from transformers import (
    WEIGHTS_NAME, get_linear_schedule_with_warmup,
    RobertaConfig, RobertaForSequenceClassification, RobertaTokenizer,
    RobertaForMaskedLM, pipeline, DataCollatorWithPadding,
    AutoModelForSequenceClassification, TrainingArguments, Trainer
)
from datasets import Dataset
import torch

!pip install evaluate
import evaluate
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 2.1 MB/s eta 0:00:00


## Transformers & Hugging Face Libraries  
- **RobertaConfig** → Configuration settings for RoBERTa models.  
- **RobertaForSequenceClassification** → RoBERTa model for classification tasks.  
- **RobertaTokenizer** → Tokenizer for RoBERTa (converts text into tokenized inputs).  
- **RobertaForMaskedLM** → RoBERTa for Masked Language Modeling (predicting masked words).  
- **pipeline** → High-level API for using pre-trained models easily.  
- **DataCollatorWithPadding** → Ensures tokenized inputs are correctly padded for training.  
- **AutoModelForSequenceClassification** → Generic method for loading classification models.  
- **TrainingArguments & Trainer** → Utilities for managing model training.  

## Torch & Optimizers  
- **torch** → PyTorch framework for training deep learning models.  
- **AdamW** → Optimizer designed for transformers.  
- **get_linear_schedule_with_warmup** → Learning rate scheduler.  

## Additional Libraries  
- **evaluate** → A package for computing accuracy, F1-score, etc., similar to `datasets.metric`.  
- **numpy & pandas** → For handling datasets and numerical operations.  
- **sklearn.train_test_split** → Splits data into training and test sets.  


In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
import pandas as pd
df=pd.read_csv('/content/drive/MyDrive/Colab_Notebooks/MSc_Fault_Tolerance/cvefixes_final.csv')

In [6]:
df.head(1)

,file_name,programming_language,code_before,code_after,diff,num_lines_added,num_lines_deleted,num_lines_in_file,num_tokens_in_file,complexity,...,commit_message,merge,cve_id,cwe_id,method_change_id,method_code,num_lines_in_method,method_complexity,num_tokens_in_method,vulnerable
0,dl-load.c,C,/* Map in a shared object's segments from the ...,/* Map in a shared object's segments from the ...,"{'added': [(152, ' const char *const start = ...",28,10,952.0,6592.0,260.0,...,Update.\n\n1999-11-09 Ulrich Drepper <dreppe...,False,CVE-1999-0199,CWE-252,217096824924488,"_dl_dst_count (const char *name, int is_path)\...",22,13,199,True


A bit of analysis to get accustomed to the new dataset.

In [7]:
print(df['cwe_id'].unique())
print(df.columns.tolist())


['CWE-252' 'SAFE' 'CWE-415' 'CWE-476' 'CWE-284' 'CWE-617' 'CWE-674'
 'CWE-190' 'CWE-400' 'CWE-416' 'CWE-835' 'CWE-665' 'CWE-369' 'CWE-404'
 'CWE-191' 'CWE-667' 'CWE-319' 'CWE-401' 'CWE-122' 'CWE-681' 'CWE-843'
 'CWE-367' 'CWE-134' 'CWE-121' 'CWE-426' 'CWE-78' 'CWE-457' 'CWE-126'
 'CWE-672' 'CWE-273' 'CWE-459' 'CWE-327']
['file_name', 'programming_language', 'code_before', 'code_after', 'diff', 'num_lines_added', 'num_lines_deleted', 'num_lines_in_file', 'num_tokens_in_file', 'complexity', 'file_change_id', 'hash', 'change_type', 'old_file_path', 'new_file_path', 'repo_url', 'author', 'committer', 'commit_message', 'merge', 'cve_id', 'cwe_id', 'method_change_id', 'method_code', 'num_lines_in_method', 'method_complexity', 'num_tokens_in_method', 'vulnerable']


Ratnaker gave me the agency to decide myself which CWEs I want to select. Therefore I want to figure out the distribution of CWE types within the dataset:

In [8]:
cwe_counts = df['cwe_id'].value_counts()
cwe_counts

,count
cwe_id,
SAFE,14066
CWE-190,687
CWE-476,471
CWE-416,421
CWE-415,171
CWE-400,161
CWE-617,142
CWE-401,84
CWE-284,78


As you can see, a lot of CWEs don't have enough representation to be used in model training and inference. Therefore, I am setting a threshold of a minimum of 50 data points required for a CWE type to be included in this model.

In [9]:
import pandas as pd

cwe_selection =  [
    'CWE-190', 'CWE-476', 'CWE-416', 'CWE-415', 'CWE-400', 'CWE-617',
    'CWE-401', 'CWE-284', 'CWE-122', 'CWE-835', 'CWE-843', 'CWE-78'
]

may_filtered_df = df[df['cwe_id'].isin(cwe_selection)]

# unique CWEs in the filtered result
unique_cwes = may_filtered_df['cwe_id'].unique()
print("Unique CWEs in the filtered dataset:", unique_cwes)

may_filtered_df.to_csv('filtered_dataset.csv', index=False)
print("Dataset has been filtered and saved as 'filtered_dataset.csv'")


Unique CWEs in the filtered dataset: ['CWE-415' 'CWE-476' 'CWE-284' 'CWE-617' 'CWE-190' 'CWE-400' 'CWE-416'
 'CWE-835' 'CWE-401' 'CWE-122' 'CWE-843' 'CWE-78']
Dataset has been filtered and saved as 'filtered_dataset.csv'


In [10]:
may_filtered_df.to_csv('/content/drive/MyDrive/Colab_Notebooks/MSc_Fault_Tolerance/23May_filtered_dataset.csv', index=False)

In [11]:
len(may_filtered_df)

2478

2478 rows in the new dataset

In [12]:
df = may_filtered_df.astype(str)

In [13]:
# Creating 2 dictionaries that convert between unique CWE types and numerical labels
id2label = dict() # Maps integer index to a CWE-type (0 : 'CWE119)
label2id = dict() # Maps CWE-type to an integer index ('CWE119' : 0)
ind = 0
for i in df['cwe_id'].unique():
    id2label[ind] = i
    label2id[i] = ind
    ind+=1

In [14]:
print('id2label dictionary: ')
print(id2label)
print('label2id dictionary: ')
print(label2id)

id2label dictionary: 
{0: 'CWE-415', 1: 'CWE-476', 2: 'CWE-284', 3: 'CWE-617', 4: 'CWE-190', 5: 'CWE-400', 6: 'CWE-416', 7: 'CWE-835', 8: 'CWE-401', 9: 'CWE-122', 10: 'CWE-843', 11: 'CWE-78'}
label2id dictionary: 
{'CWE-415': 0, 'CWE-476': 1, 'CWE-284': 2, 'CWE-617': 3, 'CWE-190': 4, 'CWE-400': 5, 'CWE-416': 6, 'CWE-835': 7, 'CWE-401': 8, 'CWE-122': 9, 'CWE-843': 10, 'CWE-78': 11}


In [15]:
df['label']=df['cwe_id'].map(label2id)
df.head()

,file_name,programming_language,code_before,code_after,diff,num_lines_added,num_lines_deleted,num_lines_in_file,num_tokens_in_file,complexity,...,merge,cve_id,cwe_id,method_change_id,method_code,num_lines_in_method,method_complexity,num_tokens_in_method,vulnerable,label
16,spnego_mech.c,C,"/*\n * Copyright (C) 2006,2008 by the Massachu...","/*\n * Copyright (C) 2006,2008 by the Massachu...","{'added': [], 'deleted': [(821, '\tgeneric_gss...",0,1,3104.0,15617.0,512.0,...,False,CVE-2014-4343,CWE-415,125656663779789,"init_ctx_reselect(OM_uint32 *minor_status, spn...",25,5,162,True,0
17,spnego_mech.c,C,"/*\n * Copyright (C) 2006,2008 by the Massachu...","/*\n * Copyright (C) 2006,2008 by the Massachu...","{'added': [(1471, '\tif (REMAIN == 0 || REMAIN...",1,1,3104.0,15621.0,513.0,...,False,CVE-2014-4344,CWE-476,165040919595628,"acc_ctx_cont(OM_uint32 *minstat,\n\t gss_b...",57,10,269,True,1
21,ldap_pwd_policy.c,C,/* -*- mode: c; c-basic-offset: 4; indent-tabs...,/* -*- mode: c; c-basic-offset: 4; indent-tabs...,"{'added': [(317, ' if (ent == NULL) {'), (3...",4,3,329.0,1998.0,54.0,...,False,CVE-2014-5353,CWE-476,153815760115106,krb5_ldap_get_password_policy_from_dn(krb5_con...,38,7,235,True,1
40,kadm_rpc_svc.c,C,"/* -*- mode: c; c-file-style: ""bsd""; indent-ta...","/* -*- mode: c; c-file-style: ""bsd""; indent-ta...","{'added': [(7, '#include <k5-int.h>'), (299, '...",3,9,269.0,1461.0,44.0,...,False,CVE-2014-9422,CWE-284,144486914514667,check_rpcsec_auth(struct svc_req *rqstp)\n{\n ...,55,9,365,True,2
130,kdc_util.c,C,/* -*- mode: c; c-basic-offset: 4; indent-tabs...,/* -*- mode: c; c-basic-offset: 4; indent-tabs...,"{'added': [(742, ' if (check_anon(kdc_activ...",1,1,1333.0,7563.0,310.0,...,False,CVE-2016-3120,CWE-476,43185298949362,validate_as_request(kdc_realm_t *kdc_active_re...,74,27,448,True,1


In [16]:
# Splitting the dataset into training(80%) and test (20%) sets
df_train, df_test = train_test_split(df, test_size=0.25, random_state=42)

It's important to check class balance in both train and test sets:

In [17]:
train_counts = df_train['cwe_id'].value_counts()
test_counts = df_test['cwe_id'].value_counts()
combined = pd.DataFrame({'train': train_counts, 'test': test_counts}).fillna(0).astype(int)
print(combined)


         train  test
cwe_id              
CWE-122     52    21
CWE-190    508   179
CWE-284     62    16
CWE-400    121    40
CWE-401     60    24
CWE-415    142    29
CWE-416    313   108
CWE-476    361   110
CWE-617     99    43
CWE-78      50    10
CWE-835     47    23
CWE-843     43    17


In [18]:
dataset = {} # Creating an empty dictionary
dataset['text'] = list(df_train['code_before']) # adding key-value pair to dataset dictionary, 'text' = key and 'code' = value (in the form of a list). Serves as the feature.
dataset['label'] = list(df_train['label']) # same, but adding the key-value pair to act as the label (prediction) for the model.
# The code below converts dictionary we just created into a Hugging Face dataset object. It provides many convenient NLP features, such as tokenization.
ds = Dataset.from_dict(dataset) # Creation of hugging face dataset object.
ds = ds.train_test_split(test_size=0.1) # train/validation split (10% validation)

The code cell above performs the **second (2ND)** data split.

### 1st Split:
* Creating the initial training and test datasets.
* test dataset is entirely separated from the training process
### 2nd Split:
* Splits the training data set into training and validation
* The validation set is used for hyperparameter tuning and intermediate evaluations during the training phase. Happens before testing

In [19]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification

tokenizer = AutoTokenizer.from_pretrained("microsoft/graphcodebert-base")

num_labels = len(label2id)

# Load model with correct classification head
model = AutoModelForSequenceClassification.from_pretrained(
    "microsoft/graphcodebert-base",
    num_labels=num_labels,
    id2label=id2label,
    label2id=label2id
)


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:104: UserWarning: 
Error while fetching `HF_TOKEN` secret value from your vault: 'Requesting secret HF_TOKEN timed out. Secrets can only be fetched when running from the Colab UI.'.
You are not authenticated with the Hugging Face Hub in this notebook.
If the error persists, please let us know by opening an issue on GitHub (https://github.com/huggingface/huggingface_hub/issues/new).
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/539 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/772 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/499M [00:00<?, ?B/s]

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at microsoft/graphcodebert-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [20]:
from sklearn.metrics import accuracy_score, precision_recall_fscore_support, confusion_matrix # model performance evaluation metrics
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)
accuracy = evaluate.load("accuracy")

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return accuracy.compute(predictions=predictions, references=labels)

# A function that calculates accuracy during model evaluation by comparing the predicted labels (after applying argmax) to the true labels.

In [21]:
def preprocess_function(examples):
    return tokenizer(examples["text"], truncation=True)

tokenized_dataset = ds.map(preprocess_function, batched=True)
# Tokenizing the dataset

Map:   0%|          | 0/1672 [00:00<?, ? examples/s]

model.safetensors:   0%|          | 0.00/499M [00:00<?, ?B/s]

Map:   0%|          | 0/186 [00:00<?, ? examples/s]

In [ ]:
from transformers import Trainer, TrainingArguments

'''
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=3,
    weight_decay=0.01,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["test"],
)

'''
training_args = TrainingArguments(
    output_dir="/content/drive/MyDrive/Colab Notebooks/THESIS_PROJECT/MODEL_WEIGHTS/NEW_MODEL_WEIGHTS/graphcodebert_bo",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=6,
    weight_decay=0.01,
    eval_strategy="epoch",
    save_strategy="epoch",
    #logging_stragety = "epoch",
    #logging_first_step = True,
    logging_steps = 1,
    load_best_model_at_end=True,
    report_to="wandb",
    fp16 = True,
    warmup_steps = 20
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["test"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

trainer.train()


<ipython-input-24-e3bcebc2644e>:40: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize?ref=models
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: alenabd24 (alenabd24-queen-mary-university-of-london) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Epoch,Training Loss,Validation Loss,Accuracy
1,1.791200,1.667984,0.489247
2,1.676400,1.421497,0.543011
3,1.498300,1.355127,0.564516
4,1.197200,1.279260,0.596774
5,0.659500,1.243351,0.639785
6,0.615700,1.212927,0.629032


TrainOutput(global_step=630, training_loss=1.2711871707250202, metrics={'train_runtime': 587.9929, 'train_samples_per_second': 17.061, 'train_steps_per_second': 1.071, 'total_flos': 2639767100129280.0, 'train_loss': 1.2711871707250202, 'epoch': 6.0})

**Saving the baseline model weights (to re-load later if necessary)

* The idea is to fine-tune the model first, so that it selects appropriate weights for the classification task.
* After training, the model's accuracy should be evaluated without bit flips
* Following that, I'll inject bit flips and compare accuracy to before vs after fault injection

In [ ]:
trainer.evaluate()

{'eval_loss': 1.2129273414611816,
 'eval_accuracy': 0.6290322580645161,
 'eval_runtime': 1.4943,
 'eval_samples_per_second': 124.475,
 'eval_steps_per_second': 8.031,
 'epoch': 6.0}

In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

preds = []
for i in df_test['code_before'].values:
    with torch.no_grad():
        inputs = tokenizer(i, return_tensors="pt",  truncation=True).to(device)
        logits = model(**inputs).logits
        predicted_class_id = logits.argmax().item()
        preds.append(predicted_class_id)

In [ ]:
y_true = [id2label[i] for i in df_test['label'].values]
y_pred = [id2label[i] for i in preds]

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(y_true, y_pred))

              precision    recall  f1-score   support

     CWE-122       0.86      0.29      0.43        21
     CWE-190       0.87      0.85      0.86       179
     CWE-284       0.92      0.69      0.79        16
     CWE-400       0.76      0.47      0.58        40
     CWE-401       0.67      0.17      0.27        24
     CWE-415       0.56      0.79      0.66        29
     CWE-416       0.53      0.64      0.58       108
     CWE-476       0.51      0.67      0.58       110
     CWE-617       0.59      0.67      0.63        43
      CWE-78       0.64      0.70      0.67        10
     CWE-835       1.00      0.09      0.16        23
     CWE-843       0.72      0.76      0.74        17

    accuracy                           0.66       620
   macro avg       0.72      0.57      0.58       620
weighted avg       0.70      0.66      0.65       620



In [ ]:
trainer.save_model("/content/drive/MyDrive/Colab_Notebooks/MSc_Fault_Tolerance/baseline_model_may")


# Changing the performance metrics:

In [22]:
import os
import json
import torch
import numpy as np
import pandas as pd
from transformers import AutoModelForSequenceClassification, AutoTokenizer
from sklearn.metrics import confusion_matrix
from IPython.display import display

# Set device
device = 'cuda' if torch.cuda.is_available() else 'cpu'

# Load model and tokenizer
model_path = "/content/drive/MyDrive/Colab_Notebooks/MSc_Fault_Tolerance/baseline_model_may"
model = AutoModelForSequenceClassification.from_pretrained(model_path).to(device)
tokenizer = AutoTokenizer.from_pretrained(model_path)

# Run inference
preds = []
model.eval()
for i in df_test['code_before'].values:
    with torch.no_grad():
        inputs = tokenizer(i, return_tensors="pt", truncation=True)
        inputs = {k: v.to(device) for k, v in inputs.items()}
        logits = model(**inputs).logits
        predicted_class_id = logits.argmax().item()
        preds.append(predicted_class_id)

# Ground truth
y_true = df_test['label'].values
y_pred = preds

# Label info
labels = list(id2label.keys())
target_names = [id2label[i] for i in labels]
cm = confusion_matrix(y_true, y_pred, labels=labels)

# Metric containers
results = []

# Per-class metrics
for i in range(len(labels)):
    TP = cm[i, i]
    FN = cm[i, :].sum() - TP
    FP = cm[:, i].sum() - TP
    TN = cm.sum() - (TP + FP + FN)
    TOTAL = TP + FN + TN + FP

    TPR = TP / (TP + FN) if (TP + FN) > 0 else 0
    FPR = FP / (FP + TN) if (FP + TN) > 0 else 0

    results.append({
        "CWE Class": target_names[i],
        "TP": TP,
        "FN": FN,
        "TN": TN,
        "FP": FP,
        "TOTAL": TOTAL,
        "TPR": TPR,
        "FPR": FPR
    })

# DataFrame
df_metrics = pd.DataFrame(results)

# Optional: add weighted average across classes
total_tp = df_metrics["TP"].sum()
total_fn = df_metrics["FN"].sum()
total_fp = df_metrics["FP"].sum()
total_tn = df_metrics["TN"].sum()
total_total = total_tp + total_fn + total_fp + total_tn

weighted_tpr = total_tp / (total_tp + total_fn) if (total_tp + total_fn) > 0 else 0
weighted_fpr = total_fp / (total_fp + total_tn) if (total_fp + total_tn) > 0 else 0

df_metrics.loc[len(df_metrics.index)] = {
    "CWE Class": "Weighted Avg",
    "TP": total_tp,
    "FN": total_fn,
    "TN": total_tn,
    "FP": total_fp,
    "TOTAL": total_total,
    "TPR": weighted_tpr,
    "FPR": weighted_fpr
}

# Display
display(df_metrics.round(3))


,CWE Class,TP,FN,TN,FP,TOTAL,TPR,FPR
0,CWE-415,23,6,573,18,620,0.793,0.030
1,CWE-476,74,36,438,72,620,0.673,0.141
2,CWE-284,11,5,603,1,620,0.688,0.002
3,CWE-617,29,14,557,20,620,0.674,0.035
4,CWE-190,152,27,419,22,620,0.849,0.050
5,CWE-400,19,21,574,6,620,0.475,0.010
6,CWE-416,69,39,452,60,620,0.639,0.117
7,CWE-835,2,21,597,0,620,0.087,0.000
8,CWE-401,4,20,594,2,620,0.167,0.003
9,CWE-122,6,15,598,1,620,0.286,0.002


True Positive Rate:
* Definition: The proportion of actual positives that were correctly identified.
* TPR = TP/ TP + FN

False Positive Rate:
* The proportion of actual negatives that were incorrectly identified as positives.

True Negative Rate:
* The proportion of actual negatives that were correctly identified as negatives.

False Negative Rate:
* The proportion of actual positives that were incorrectly classified as negatives.

# Bit Flipping

The function below flips each bit in every weight of hte query layer with a certain assigned probability 0< x <1

| Weight Tensor     | Role in Attention | Effect of Corruption / Bit Flips                                                                    |
| ----------------- | ----------------- | --------------------------------------------------------------------------------------------------- |
| `query.weight`    | Computes Q        | Affects **what** each token focuses on. Likely to alter the attention map structure.                |
| `key.weight`      | Computes K        | Affects **how well** the query aligns with each token. Impacts **precision** of focus.              |
| `value.weight`    | Computes V        | Directly alters **information passed through attention**. May affect final representations heavily. |
| `out_proj.weight` | Combines outputs  | Affects how different head outputs are **merged**. Can introduce noise globally across the model.   |


In [23]:
import os
import json
import torch
import random
import struct
import numpy as np
import pandas as pd
from transformers import AutoModelForSequenceClassification, AutoTokenizer
from sklearn.metrics import confusion_matrix
from IPython.display import display

def flip_bits_with_probability(model, flip_prob=0.001):
    flipped = []
    for layer_index, layer in enumerate(model.roberta.encoder.layer):
        weight = layer.attention.self.query.weight
        weight_data = weight.data.cpu().numpy()
        num_rows, num_cols = weight_data.shape

        for row in range(num_rows):
            for col in range(num_cols):
                original_val = weight_data[row, col]
                int_bits = struct.unpack('>I', struct.pack('>f', original_val))[0]
                flipped_bits = int_bits

                for bit_idx in range(32):
                    if random.random() < flip_prob:
                        flipped_bits ^= (1 << bit_idx)

                if flipped_bits != int_bits:
                    flipped_val = struct.unpack('>f', struct.pack('>I', flipped_bits))[0]
                    weight_data[row, col] = flipped_val
                    flipped.append((layer_index, row, col, original_val, flipped_val))

        weight.data = torch.tensor(weight_data, dtype=weight.dtype, device=weight.device)

    return flipped


def evaluate_model(model, tokenizer, df_test, id2label, save_path):
    device = 'cuda' if torch.cuda.is_available() else 'cpu'
    preds = []
    model.eval()

    for i in df_test['code_before'].values:
        with torch.no_grad():
            inputs = tokenizer(i, return_tensors="pt", truncation=True)
            inputs = {k: v.to(device) for k, v in inputs.items()}
            logits = model(**inputs).logits
            predicted_class_id = logits.argmax().item()
            preds.append(predicted_class_id)

    y_true = df_test['label'].values
    y_pred = preds

    labels = list(id2label.keys())
    target_names = [id2label[i] for i in labels]
    cm = confusion_matrix(y_true, y_pred, labels=labels)

    # Initialize lists to collect per-class metrics
    TP_list, FN_list, TN_list, FP_list, TOTAL_list, TPR_list, FPR_list = [], [], [], [], [], [], []

    for i in range(len(labels)):
        TP = cm[i, i]
        FN = cm[i, :].sum() - TP
        FP = cm[:, i].sum() - TP
        TN = cm.sum() - (TP + FP + FN)
        total = TP + FN + TN + FP

        TP_list.append(TP)
        FN_list.append(FN)
        TN_list.append(TN)
        FP_list.append(FP)
        TOTAL_list.append(total)
        TPR_list.append(TP / (TP + FN) if (TP + FN) > 0 else 0)
        FPR_list.append(FP / (FP + TN) if (FP + TN) > 0 else 0)

    df_metrics = pd.DataFrame({
        "CWE Class": target_names,
        "TP": TP_list,
        "FN": FN_list,
        "TN": TN_list,
        "FP": FP_list,
        "TOTAL": TOTAL_list,
        "TPR": TPR_list,
        "FPR": FPR_list
    })

    # Weighted average
    total_support = np.array(TP_list) + np.array(FN_list)
    total_all = np.sum(TOTAL_list)

    weighted_avg = {
        "CWE Class": "Weighted Avg",
        "TP": np.sum(TP_list),
        "FN": np.sum(FN_list),
        "TN": np.sum(TN_list),
        "FP": np.sum(FP_list),
        "TOTAL": total_all,
        "TPR": np.sum(TP_list) / (np.sum(TP_list) + np.sum(FN_list)) if (np.sum(TP_list) + np.sum(FN_list)) > 0 else 0,
        "FPR": np.sum(FP_list) / (np.sum(FP_list) + np.sum(TN_list)) if (np.sum(FP_list) + np.sum(TN_list)) > 0 else 0
    }

    df_metrics = pd.concat([df_metrics, pd.DataFrame([weighted_avg])], ignore_index=True)
    df_metrics.to_csv(save_path, index=False)
    display(df_metrics.round(3))


In [25]:
import os
import json
import torch
import random
import struct
import numpy as np
import pandas as pd
from transformers import AutoModelForSequenceClassification, AutoTokenizer
from sklearn.metrics import confusion_matrix
from IPython.display import display
from collections import defaultdict

base_model_path = "/content/drive/MyDrive/Colab_Notebooks/MSc_Fault_Tolerance/baseline_model_may"
results_dir = os.path.join(base_model_path, "probability_flip_results_2")
os.makedirs(results_dir, exist_ok=True)

flip_probs = [
    0.0,
    1e-6, 2e-6, 3e-6, 4e-6, 5e-6, 6e-6, 7e-6, 8e-6, 9e-6,
    1e-5, 2e-5, 3e-5, 4e-5, 5e-5, 6e-5, 7e-5, 8e-5, 9e-5
]
save_counts = defaultdict(int)

for flip_prob in flip_probs:
    print(f"\n🌀 Flip Prob = {flip_prob:.0e} --- Running Experiment")

    model = AutoModelForSequenceClassification.from_pretrained(base_model_path).to(device)
    tokenizer = AutoTokenizer.from_pretrained(base_model_path)

    flip_bits_with_probability(model, flip_prob=flip_prob)

    save_counts[flip_prob] += 1
    filename = f"metrics_p{flip_prob:.0e}_run{save_counts[flip_prob]}.csv"
    save_path = os.path.join(results_dir, filename)

    # evaluation and save
    evaluate_model(model, tokenizer, df_test, id2label, save_path)

    print(f"✅ Saved metrics to {filename}")



🌀 Flip Prob = 0e+00 --- Running Experiment


,CWE Class,TP,FN,TN,FP,TOTAL,TPR,FPR
0,CWE-415,23,6,573,18,620,0.793,0.030
1,CWE-476,74,36,438,72,620,0.673,0.141
2,CWE-284,11,5,603,1,620,0.688,0.002
3,CWE-617,29,14,557,20,620,0.674,0.035
4,CWE-190,152,27,419,22,620,0.849,0.050
5,CWE-400,19,21,574,6,620,0.475,0.010
6,CWE-416,69,39,452,60,620,0.639,0.117
7,CWE-835,2,21,597,0,620,0.087,0.000
8,CWE-401,4,20,594,2,620,0.167,0.003
9,CWE-122,6,15,598,1,620,0.286,0.002


✅ Saved metrics to metrics_p0e+00_run1.csv

🌀 Flip Prob = 1e-06 --- Running Experiment


,CWE Class,TP,FN,TN,FP,TOTAL,TPR,FPR
0,CWE-415,17,12,580,11,620,0.586,0.019
1,CWE-476,76,34,406,104,620,0.691,0.204
2,CWE-284,11,5,603,1,620,0.688,0.002
3,CWE-617,22,21,566,11,620,0.512,0.019
4,CWE-190,151,28,416,25,620,0.844,0.057
5,CWE-400,15,25,576,4,620,0.375,0.007
6,CWE-416,69,39,443,69,620,0.639,0.135
7,CWE-835,0,23,597,0,620,0.000,0.000
8,CWE-401,4,20,596,0,620,0.167,0.000
9,CWE-122,3,18,598,1,620,0.143,0.002


✅ Saved metrics to metrics_p1e-06_run1.csv

🌀 Flip Prob = 2e-06 --- Running Experiment


...

✅ Saved metrics to metrics_p7e-05_run1.csv

🌀 Flip Prob = 8e-05 --- Running Experiment


,CWE Class,TP,FN,TN,FP,TOTAL,TPR,FPR
0,CWE-415,29,0,0,591,620,1.000,1.000
1,CWE-476,0,110,510,0,620,0.000,0.000
2,CWE-284,0,16,604,0,620,0.000,0.000
3,CWE-617,0,43,577,0,620,0.000,0.000
4,CWE-190,0,179,441,0,620,0.000,0.000
5,CWE-400,0,40,580,0,620,0.000,0.000
6,CWE-416,0,108,512,0,620,0.000,0.000
7,CWE-835,0,23,597,0,620,0.000,0.000
8,CWE-401,0,24,596,0,620,0.000,0.000
9,CWE-122,0,21,599,0,620,0.000,0.000


✅ Saved metrics to metrics_p8e-05_run1.csv

🌀 Flip Prob = 9e-05 --- Running Experiment


,CWE Class,TP,FN,TN,FP,TOTAL,TPR,FPR
0,CWE-415,29,0,0,591,620,1.000,1.000
1,CWE-476,0,110,510,0,620,0.000,0.000
2,CWE-284,0,16,604,0,620,0.000,0.000
3,CWE-617,0,43,577,0,620,0.000,0.000
4,CWE-190,0,179,441,0,620,0.000,0.000
5,CWE-400,0,40,580,0,620,0.000,0.000
6,CWE-416,0,108,512,0,620,0.000,0.000
7,CWE-835,0,23,597,0,620,0.000,0.000
8,CWE-401,0,24,596,0,620,0.000,0.000
9,CWE-122,0,21,599,0,620,0.000,0.000


✅ Saved metrics to metrics_p9e-05_run1.csv


# New bit flipping strategy 5th June 2025 (Lev's reccomendation)

Proposed Approach:
Assign a bit-flip probability (p) to each bit in the model's weights. For each bit, you decide whether to flip it based on this probability. For instance, with p = 0.001, each bit has a 0.1% chance of being flipped.

Implementation Steps:

1. Iterate over each weight in the target layer.

2. Convert the weight to its 32-bit binary representation.

3. For each bit in the binary representation:

* Generate a random number between 0 and 1.

* If this number is less than p, flip the bit.

4. Convert the modified binary back to a floating-point number.

5. Update the weight with the new value.

This method ensures that bit flips are distributed throughout the model in a manner consistent with how faults might naturally occur in hardware.

tokenize the subset as earlier: